## Importing dependencies

In [23]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import  StandardScaler
from sklearn.model_selection import  train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  accuracy_score

## Data collection and Analysis

PIMA Diabetes Dataset

In [2]:
def load_data(path):
  return pd.read_csv(path)

In [5]:
# loading data with pandas
diabetes_dataset = load_data('diabetes.csv')
diabetes_dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
# number of rows and columns in the dataset
diabetes_dataset.shape

(768, 9)

In [7]:
# getting statistical measures of data
diabetes_dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [8]:
diabetes_dataset.Outcome.value_counts()

,count
Outcome,
0,500
1,268


0 --> Non diabetic

1 --> Diabetic

In [9]:
diabetes_dataset.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [10]:
# Separating features and labels
X = diabetes_dataset.drop(axis=1, columns='Outcome')
y = diabetes_dataset.Outcome

In [11]:
X.shape, y.shape

((768, 8), (768,))

## Data Standardization

In [12]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

## Data Splitting

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [18]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((614, 8), (614,), (154, 8), (154,))

## Training models

In [28]:
svm_model = SVC()
svm_model.fit(X_train, y_train)

SVC()

In [24]:
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train, y_train)

LogisticRegression()

## Validation model

In [21]:
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [29]:
val_score = accuracy_score(svm_model.predict(X_valid), y_valid)
train_score = accuracy_score(svm_model.predict(X_train), y_train)
print(f'Validation score: {val_score}')
print(f'Train score: {train_score}')

Validation score: 0.7272727272727273
Train score: 0.8289902280130294


In [30]:
val_score = accuracy_score(log_reg_model.predict(X_valid), y_valid)
train_score = accuracy_score(log_reg_model.predict(X_train), y_train)
print(f'Validation score: {val_score}')
print(f'Train score: {train_score}')

Validation score: 0.7272727272727273
Train score: 0.7703583061889251


## Model fine tunning

In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 500, 1000]
}

grid_search = GridSearchCV(log_reg_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'max_iter': [100, 500, 1000], 'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']},
             scoring='accuracy')

In [33]:
print(grid_search.best_params_)

{'C': 0.1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'saga'}


In [34]:
model = grid_search.best_estimator_

## Model assessment

In [35]:
train_score = accuracy_score(model.predict(X_train), y_train)
test_score = accuracy_score(model.predict(X_test), y_test)
print(f'Train score: {train_score}')
print(f'Test score: {test_score}')

Train score: 0.7671009771986971
Test score: 0.7922077922077922


## Prediction

In [36]:
def predict_diabetes(input_data):
  input_data = np.asarray(input_data)
  input_data = input_data.reshape(1, -1)
  input_data = scaler.transform(input_data)
  prediction = model.predict(input_data)

  if prediction[0] == 0:
    print("The person is not diabetic")
  else:
    print("The person is diabetic")

In [38]:
data = [6, 148, 72, 35, 0, 33.6, 0.627, 50]
predict_diabetes(data)

The person is diabetic


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
